<a href="https://www.kaggle.com/code/mafiosoquasar/time-series-sales-forecasting-with-pytorch?scriptVersionId=210967423" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import argparse
import os
import random,numpy
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [2]:
pip install pandas 


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchvision.transforms.functional as RF
from torchvision.models.feature_extraction import create_feature_extractor
from PIL import Image
import numpy as np
import random,cv2,pandas,os,numpy
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn.preprocessing import OneHotEncoder

In [4]:
seed = 999
print("Random Seed: ", seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

nz = 100
beta1 = 0.5
lr = 0.0001
batch_size=100
ngpu=1
ngf,nc = 3,3
ndf = 64
shape=(308263,10)

device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

Random Seed:  999


In [5]:
train = pandas.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_train.csv', parse_dates=['date'])
test = pandas.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_test.csv', parse_dates=['date'])
ss = pandas.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/solution.csv')

inventory = pandas.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/inventory.csv')
weights = pandas.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/test_weights.csv')
calendar = pandas.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/calendar.csv', parse_dates=['date'])

Frankfurt_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Frankfurt_1"')
Prague_2 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_2"')
Brno_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Brno_1"')
Munich_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Munich_1"')
Prague_3 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_3"')
Prague_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_1"')
Budapest_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Budapest_1"')

def process_calendar(df):

    df = df.sort_values('date').reset_index(drop=True)

    df['next_holiday_date'] = df.loc[df['holiday'] == 1, 'date'].shift(-1)
    df['next_holiday_date'] = df['next_holiday_date'].bfill()
    df['days_to_holiday'] = (df['next_holiday_date'] - df['date']).dt.days
    df.drop(columns=['next_holiday_date'], inplace=True)

    df['next_shops_closed_date'] = df.loc[df['shops_closed'] == 1, 'date'].shift(-1)
    df['next_shops_closed_date'] = df['next_shops_closed_date'].bfill()
    df['days_to_shops_closed'] = (df['next_shops_closed_date'] - df['date']).dt.days
    df.drop(columns=['next_shops_closed_date'], inplace=True)

    df['day_after_closing'] = (
        (df['shops_closed'] == 0) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)

    df['long_weekend'] = (
        (df['shops_closed'] == 1) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)

    df['weekday'] = df['date'].dt.weekday 
    return df

dfs = ['Frankfurt_1', 'Prague_2', 'Brno_1', 'Munich_1', 'Prague_3', 'Prague_1', 'Budapest_1']

processed_dfs = [process_calendar(globals()[df]) for df in dfs]

calendar_extended = pandas.concat(processed_dfs).sort_values('date').reset_index(drop=True)

train_calendar = train.merge(calendar_extended, on=['date', 'warehouse'], how='left')
train_inventory = train_calendar.merge(inventory, on=['unique_id', 'warehouse'], how='left')
train_data = train_inventory.merge(weights, on=['unique_id'], how='left')

test_calendar = test.merge(calendar_extended, on=['date', 'warehouse'], how='left')
test_data = test_calendar.merge(inventory, on=['unique_id', 'warehouse'], how='left')

train_data.dropna(subset=['sales'], inplace=True)
train_data = train_data.sort_values(['unique_id', 'date'])

train_data.head()

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,day_after_closing,long_weekend,weekday,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,weight
3910978,0,2022-07-18,Budapest_1,5289.0,3.97,710.89,0.09,0.0,0.0,0.00000,...,0,0,0,0,Plum_0,Fruit and vegetable,Fruit and vegetable_L2_0,Fruit and vegetable_L3_0,Fruit and vegetable_L4_0,2.535369
3910983,0,2022-07-19,Budapest_1,5255.0,73.36,710.89,1.00,0.0,0.0,0.00000,...,0,0,1,0,Plum_0,Fruit and vegetable,Fruit and vegetable_L2_0,Fruit and vegetable_L3_0,Fruit and vegetable_L4_0,2.535369
3910959,0,2022-07-20,Budapest_1,5334.0,558.09,710.89,0.96,0.0,0.0,0.45045,...,0,0,2,0,Plum_0,Fruit and vegetable,Fruit and vegetable_L2_0,Fruit and vegetable_L3_0,Fruit and vegetable_L4_0,2.535369
3910961,0,2022-07-21,Budapest_1,5459.0,14.03,710.89,0.06,0.0,0.0,0.45045,...,0,0,3,0,Plum_0,Fruit and vegetable,Fruit and vegetable_L2_0,Fruit and vegetable_L3_0,Fruit and vegetable_L4_0,2.535369
3910956,0,2022-07-22,Budapest_1,5461.0,558.53,710.89,0.97,0.0,0.0,0.45045,...,0,0,4,0,Plum_0,Fruit and vegetable,Fruit and vegetable_L2_0,Fruit and vegetable_L3_0,Fruit and vegetable_L4_0,2.535369


In [6]:
train_data_ = train_data.drop(columns=['availability'])
x = train_data_.drop(columns=['sales'])

categorical_columns = x.select_dtypes(include=['object']).columns.tolist()
encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(x[categorical_columns])

In [7]:
x = torch.from_numpy(one_hot_encoded).type('torch.FloatTensor')
y = torch.from_numpy(train_data_['sales'].to_numpy()).type('torch.FloatTensor')

In [8]:
x.shape, y.shape, x.shape

(torch.Size([4007367, 2991]),
 torch.Size([4007367]),
 torch.Size([4007367, 2991]))

In [9]:
train_x = torch.utils.data.DataLoader(x,batch_size=batch_size)
train_y = torch.utils.data.DataLoader(y,batch_size=batch_size)

In [10]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data, gain=0.02)
        
class EffnetModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.rafire_1 = nn.LSTMCell(2991, 1524)
        self.rafire_1_b = nn.Sequential(nn.BatchNorm1d(1524),
                                        nn.ReLU())

        self.rafire_2 = nn.LSTMCell(1524, 1324)
        self.rafire_2_b = nn.Sequential(nn.BatchNorm1d(1324),
                                        nn.ReLU())

        self.rafire_3 = nn.LSTMCell(1324, 1024)
        self.rafire_3_b = nn.Sequential(nn.BatchNorm1d(1024),
                                        nn.ReLU())

        self.rafire_4 = nn.LSTMCell(1024, 824)
        self.rafire_4_b = nn.Sequential(nn.BatchNorm1d(824),
                                        nn.ReLU())

        self.rafire_5 = nn.LSTMCell(824, 524)
        self.rafire_5_b = nn.Sequential(nn.BatchNorm1d(524),
                                        nn.ReLU())

        self.rafire_6 = nn.LSTMCell(524, 324)
        self.rafire_6_b = nn.Sequential(nn.BatchNorm1d(324),
                                        nn.ReLU())

        self.rafire_7 = nn.LSTMCell(324, 124)
        self.rafire_7_b = nn.Sequential(nn.BatchNorm1d(124),
                                        nn.ReLU())
            
        self.rafire_8 = nn.Linear(124, 1)

    def forward(self,x):
        x,_=self.rafire_1(x)
        x=self.rafire_1_b(x)
        x,_=self.rafire_2(x)
        x=self.rafire_2_b(x)
        x,_=self.rafire_3(x)
        x=self.rafire_3_b(x)
        x,_=self.rafire_4(x)
        x=self.rafire_4_b(x)
        x,_=self.rafire_5(x)
        x=self.rafire_5_b(x)
        x,_=self.rafire_6(x)
        x=self.rafire_6_b(x)
        x,_=self.rafire_7(x)
        x=self.rafire_7_b(x)
        
        return self.rafire_8(x)

In [11]:
EFF_NET = EffnetModel().float()
EFF_NET= nn.DataParallel(EFF_NET).to(device)

In [12]:
criterion = nn.L1Loss()

optimizer = optim.AdamW(EFF_NET.parameters(), lr=lr,betas=(beta1, 0.999))
scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.86)

In [ ]:
high_rf,i,j_r,k,z_w_=100000,0,0,0,[]
    
correct,total=0,0

while(True):
    optimizer.zero_grad()
    for data,label in zip(train_x,train_y):
        #z_ = nn.Embedding(10, 3, dtype=torch.float64)
        #print(z_(torch.FloatTensor(data)))
        output = EFF_NET(data.to(device)).view(-1)
        err_real = criterion(output, label.to(device))
        err_real.backward()
        optimizer.step()
        #print(err_real.item())
        
    print(f"EPOCH : {i} LOSS_wl : {err_real.item()}")

    if len(z_w_)>=3:
        if len([True for i in range(1,4) if z_w_[len(z_w_)-i]<=z_w_[len(z_w_)-3] and z_w_[len(z_w_)-i]>=z_w_[len(z_w_)-4]])==3:
            z_w_,j_r=[],0
            print(f"lr_br:{optimizer.param_groups[0]['lr']}".upper())
            scheduler.step()
            print(f"lr_up:{optimizer.param_groups[0]['lr']}".upper())
            
    z_w_.append(err_real.item())
    if err_real.item()<high_rf:
            high_rf=err_real.item()
            torch.save(EFF_NET.state_dict(),f'/kaggle/working/{err_real.item()}.pth')
    if i==50:
        break
    i+=1

12463.6904296875
4814.01220703125
922.1072387695312
29407.943359375
3748.300537109375
905.4717407226562
900.9874877929688
1554.265869140625
150863.515625
20200.8359375
3217.416015625
1375.5511474609375
1367.65185546875
3932.513916015625
1078.2877197265625
940.17919921875
2329.59765625
6484.85400390625
1118.711669921875
2390.55712890625
3790.560302734375
4010.958740234375
6024.2021484375
9561.203125
65232.18359375
36802.265625
12819.15625
12299.875
47811.15234375
7876.77490234375
3221.119384765625
11051.421875
9100.2919921875
7398.1416015625
2477.477294921875
13946.0517578125
24122.775390625
7629.9228515625
14678.849609375
22304.001953125
2136.614013671875
489.0760498046875
457.1211242675781
950.9667358398438
2493.11474609375
54305.78515625
1790.5440673828125
448.10699462890625
4381.544921875
3875.477294921875
201.88949584960938
115.65766906738281
21674.62109375
63919.79296875
26844.46875
575.2731323242188
205.40298461914062
369.7756042480469
2055.07763671875
681.5108642578125
2577.5378